# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, make_scorer
import torch
import transformers
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
import lightgbm as lgbm
import catboost as catboost

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


В таблице есть две столбца: text, собственно текст комментария, и toxic, который содержит целевой признак по сентименту фразы. Поработаем с датасетом,произведём стандратные шаги на выявление ошибок.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
df.isnull().sum()

text     0
toxic    0
dtype: int64

In [5]:
df.duplicated().sum()

0

In [6]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Пропусков, дубликатов - не обнаружено, поэтому выделяем корпус данных для обучения модели из обучающей выборки.

In [7]:
corpus = list(df['text'])
corpus[0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

Заведем функцию для очистки текста.

In [8]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    return text

Прогоняем через эту функцию циклом весь наш текст.

In [9]:
for i in range(len(corpus)):
    corpus[i] = clear_text(corpus[i])
    
print(corpus[0])

Explanation Why the edits made under my username Hardcore Metallica Fan were reverted  They weren t vandalisms  just closure on some GAs after I voted at New York Dolls FAC  And please don t remove the template from the talk page since I m retired now             


Разбиваем фразу на слагаемые, то есть, слова.

In [10]:
for i in range(len(corpus)):
    corpus[i] = nltk.word_tokenize(corpus[i])
    
print(corpus[0])

['Explanation', 'Why', 'the', 'edits', 'made', 'under', 'my', 'username', 'Hardcore', 'Metallica', 'Fan', 'were', 'reverted', 'They', 'weren', 't', 'vandalisms', 'just', 'closure', 'on', 'some', 'GAs', 'after', 'I', 'voted', 'at', 'New', 'York', 'Dolls', 'FAC', 'And', 'please', 'don', 't', 'remove', 'the', 'template', 'from', 'the', 'talk', 'page', 'since', 'I', 'm', 'retired', 'now']


Заводим функцию лемматизации слова в тело кода.

In [11]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    lemm_list = lemmatizer.lemmatize(text)
    return lemm_list

Вновь прогон]ем весь текст через функцию леммы.

In [12]:
for i in range(len(corpus)):
    for n in range(len(corpus[i])):
        corpus[i][n] = lemmatize(corpus[i][n])        

print(corpus[0])

['Explanation', 'Why', 'the', 'edits', 'made', 'under', 'my', 'username', 'Hardcore', 'Metallica', 'Fan', 'were', 'reverted', 'They', 'weren', 't', 'vandalism', 'just', 'closure', 'on', 'some', 'GAs', 'after', 'I', 'voted', 'at', 'New', 'York', 'Dolls', 'FAC', 'And', 'please', 'don', 't', 'remove', 'the', 'template', 'from', 'the', 'talk', 'page', 'since', 'I', 'm', 'retired', 'now']


Теперь джойним обратно.

In [13]:
for i in range(len(corpus)):
    corpus[i] = " ".join(corpus[i])
    
print(corpus[0])

Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They weren t vandalism just closure on some GAs after I voted at New York Dolls FAC And please don t remove the template from the talk page since I m retired now


Вводим стоп-слова, создаем счетчик, уоторый превратит текст в мешок слов.

In [14]:
nltk.download('stopwords') 
stop_words = set(stopwords.words('english')) 
count_vect = CountVectorizer()

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Передаем корпуск текстов через фит.

In [15]:
bow = count_vect.fit_transform(corpus)

In [16]:
print("Размер мешка без учёта стоп-слов:", bow.shape)

Размер мешка без учёта стоп-слов: (159571, 164412)


Теперь передадим счетчику корпус текстов с учетом стоп-слов.

In [17]:
count_vect = CountVectorizer(stop_words=stop_words)
bow = count_vect.fit_transform(corpus)

Считаем TF-IDF.

In [18]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words) 
tf_idf = count_tf_idf.fit_transform(corpus)

In [19]:
print("Размер матрицы:", tf_idf.shape)

Размер матрицы: (159571, 164267)


Разделим данные на обучающую, тестовую и валидационную.

In [20]:
features, features_test, target, target_test = train_test_split(tf_idf, 
                                    df.toxic, test_size = 0.2, train_size = 0.8)
features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                    target, test_size = 0.25, train_size = 0.75)

Данные готовы, поехали их обучать и смотреть показатели метрики F1, главной для нас метрики в этом исследовании.

## Обучение

### LogisticRegression

In [21]:
model = LogisticRegression(random_state=12345)

In [22]:
f1 = make_scorer(f1_score , average='macro')

In [23]:
parametrs_lr = { 'C': range (1, 10, 1),
              'class_weight': ['balanced'],
              }

grid = GridSearchCV(model, parametrs_lr, scoring = 'f1', cv=5)
grid.fit(features_train, target_train)
grid.best_params_

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


{'C': 5, 'class_weight': 'balanced'}

In [24]:
model = LogisticRegression(random_state=12345, C=8, class_weight='balanced')
model.fit(features_train, target_train)

LogisticRegression(C=8, class_weight='balanced', dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
pred = model.predict(features_test)
print('Logistic Regression F1_score', f1_score(target_test, pred))
print('Logistic Regression accuracy_score', accuracy_score(target_test, pred))

Logistic Regression F1_score 0.7537541915731155
Logistic Regression accuracy_score 0.9470781764060786


In [26]:
print('F1_score', f1_score(target_test, pred))

F1_score 0.7537541915731155


### LGBMClassifier

In [27]:
lgb = lgbm.LGBMClassifier()

params_lgb = {'n_estimators': range (1, 10, 2),
              'learning_rate': [0.1, 0.9],
             }
 
grid_lgb = GridSearchCV(lgb, params_lgb, scoring = 'f1', cv=5)

grid_lgb.fit(features_train, target_train)

grid_lgb.best_params_

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


{'learning_rate': 0.9, 'n_estimators': 1}

In [28]:
lgb = lgbm.LGBMClassifier(n_estimators = 1, learning_rate = 0.9)

lgb.fit(features_train, target_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.9, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [29]:
predictions_lgb = lgb.predict(features_test)
print('LGB F1_score', f1_score(target_test, predictions_lgb))
print('LGB accuracy_score', accuracy_score(target_test, predictions_lgb))

LGB F1_score 0.6868863955119213
LGB accuracy_score 0.9440388532038226


### CatBoost

In [30]:
ctb = catboost.CatBoostClassifier()

params_ctb = {'n_estimators': range (1, 10, 5),
              'learning_rate': [0.1, 0.9],
             }
 
grid_ctb = GridSearchCV(ctb, params_ctb, scoring = 'f1', cv=5)

grid_ctb.fit(features_train, target_train)

grid_ctb.best_params_

0:	learn: 0.5917785	total: 3.79s	remaining: 0us
0:	learn: 0.5944247	total: 3.74s	remaining: 0us
0:	learn: 0.5947999	total: 3.82s	remaining: 0us
0:	learn: 0.5937068	total: 3.89s	remaining: 0us
0:	learn: 0.5938254	total: 4.04s	remaining: 0us
0:	learn: 0.5917785	total: 3.93s	remaining: 19.7s
1:	learn: 0.5159328	total: 7.13s	remaining: 14.3s
2:	learn: 0.4566199	total: 10.3s	remaining: 10.3s
3:	learn: 0.4086647	total: 13.6s	remaining: 6.82s
4:	learn: 0.3722430	total: 16.9s	remaining: 3.39s
5:	learn: 0.3452738	total: 20.2s	remaining: 0us
0:	learn: 0.5944247	total: 3.96s	remaining: 19.8s
1:	learn: 0.5177326	total: 7.34s	remaining: 14.7s
2:	learn: 0.4571144	total: 10.6s	remaining: 10.6s
3:	learn: 0.4096678	total: 13.7s	remaining: 6.87s
4:	learn: 0.3720982	total: 16.9s	remaining: 3.39s
5:	learn: 0.3438517	total: 20s	remaining: 0us
0:	learn: 0.5947999	total: 3.82s	remaining: 19.1s
1:	learn: 0.5175437	total: 7.02s	remaining: 14s
2:	learn: 0.4556282	total: 10.3s	remaining: 10.3s
3:	learn: 0.409632

{'learning_rate': 0.9, 'n_estimators': 6}

In [31]:
ctb = catboost.CatBoostClassifier(n_estimators = 6, learning_rate = 0.9)

ctb.fit(features_train, target_train)

0:	learn: 0.2603912	total: 4.64s	remaining: 23.2s
1:	learn: 0.2325912	total: 8.34s	remaining: 16.7s
2:	learn: 0.2161930	total: 12.2s	remaining: 12.2s
3:	learn: 0.2053360	total: 16.2s	remaining: 8.11s
4:	learn: 0.1925677	total: 20.3s	remaining: 4.07s
5:	learn: 0.1880770	total: 24.2s	remaining: 0us


In [32]:
predictions_ctb = ctb.predict(features_test)
print('CatBoost F1_score', f1_score(target_test, predictions_ctb))
print('CatBoost accuracy_score', accuracy_score(target_test, predictions_ctb))

CatBoost F1_score 0.6475856697819314
CatBoost accuracy_score 0.943286855710481


## Выводы

Лучшую метрику F1 мы получили на модели LogReg; LightGBM, Catboost - показали метрику F1 меньшую, чем требуется. Что нового я узнала из нынешнего исследования? А вот что - для успешного выполнения работы потребовалось найти и занять на время ноут с опперативкой в 8Гб, процессором мощностью в 3,1 и подорожник. Потому что подорожник всегда поможет, это известно с детства.